In [67]:
import pandas as pd
import numpy as np

In [68]:
sen = pd.read_csv('./datasets/sentiment_words_simple.csv')
sen.head()

,pos,word,pos_score,neg_score
0,adj,.22-caliber,0.0,0.0
1,adj,.22-calibre,0.0,0.0
2,adj,.22_caliber,0.0,0.0
3,adj,.22_calibre,0.0,0.0
4,adj,.38-caliber,0.0,0.0


In [69]:
# objectivity score = 1-pos_score-neg_score
sen['objectivity_score']=1-sen.pos_score - sen.neg_score
sen.head()

,pos,word,pos_score,neg_score,objectivity_score
0,adj,.22-caliber,0.0,0.0,1.0
1,adj,.22-calibre,0.0,0.0,1.0
2,adj,.22_caliber,0.0,0.0,1.0
3,adj,.22_calibre,0.0,0.0,1.0
4,adj,.38-caliber,0.0,0.0,1.0


In [70]:
# load in movie review dataset
rt= pd.read_csv('./datasets/rt_critics.csv')
rt.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title
0,Derek Adams,fresh,114709.0,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559.0,Toy story
1,Richard Corliss,fresh,114709.0,TIME Magazine,The year's most inventive comedy.,2008-08-31,9559.0,Toy story
2,David Ansen,fresh,114709.0,Newsweek,A winning animated feature that has something ...,2008-08-18,9559.0,Toy story
3,Leonard Klady,fresh,114709.0,Variety,The film sports a provocative and appealing st...,2008-06-09,9559.0,Toy story
4,Jonathan Rosenbaum,fresh,114709.0,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10,9559.0,Toy story


In [71]:
rt.shape

(14072, 8)

In [72]:
#load clean dataset
rt=pd.read_csv('./datasets/rt_critics_prep_done.csv')
rt.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title,quote_len,qt
0,Derek Adams,1,114709.0,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559.0,Toy story,24,so ingenious in concept design and execution t...
1,David Ansen,1,114709.0,Newsweek,A winning animated feature that has something ...,2008-08-18,9559.0,Toy story,13,a winning animated feature that has something ...
2,Leonard Klady,1,114709.0,Variety,The film sports a provocative and appealing st...,2008-06-09,9559.0,Toy story,17,the film sports a provocative and appealing st...
3,Jonathan Rosenbaum,1,114709.0,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10,9559.0,Toy story,14,an entertaining computer generated hyperrealis...
4,Michael Booth,1,114709.0,Denver Post,"As Lion King did before it, Toy Story revived ...",2007-05-03,9559.0,Toy story,40,as lion king did before it toy story revived t...


In [73]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [74]:
quote=rt.qt[0]
quote

'so ingenious in concept design and execution that you could watch it on a postage stamp sized screen and still be engulfed by its charm'

In [75]:
type(quote)

str

In [76]:
doc = nlp(quote)
doc

so ingenious in concept design and execution that you could watch it on a postage stamp sized screen and still be engulfed by its charm

In [77]:
ingenious=doc[1]

In [78]:
ingenious.pos_

'ADJ'

In [79]:
ingenious.dep_

'ROOT'

In [80]:
from tqdm import tqdm
parsed_quotes = []
for i, parsed in tqdm(enumerate(nlp.pipe(rt.qt.values, batch_size=50, n_threads=4))):
    assert parsed.is_parsed
    parsed_quotes.append(parsed)

11215it [00:58, 190.10it/s]


In [81]:
unique_pos = []
for parsed in parsed_quotes:
    unique_pos.extend([t.pos_ for t in parsed])
unique_pos = np.unique(unique_pos)
print(unique_pos)

['ADJ' 'ADP' 'ADV' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NUM' 'PART' 'PRON' 'PROPN'
 'PUNCT' 'SPACE' 'SYM' 'VERB' 'X']


In [82]:
for pos in unique_pos:
    rt[pos+'_prop'] = 0

In [83]:
rt = rt.reset_index(drop=True)
for i, parsed in tqdm(enumerate(parsed_quotes)):
    parsed_len = len(parsed)
    for pos in unique_pos:
        count = len([x for x in parsed if x.pos_ == pos])
        rt.loc[i, pos+'_prop'] = float(count)/parsed_len

11215it [01:28, 126.44it/s]


In [84]:
rt.head()

,critic,fresh,imdb,publication,quote,review_date,rtid,title,quote_len,qt,...,NOUN_prop,NUM_prop,PART_prop,PRON_prop,PROPN_prop,PUNCT_prop,SPACE_prop,SYM_prop,VERB_prop,X_prop
0,Derek Adams,1,114709.0,Time Out,"So ingenious in concept, design and execution ...",2009-10-04,9559.0,Toy story,24,so ingenious in concept design and execution t...,...,0.280000,0.0,0.00000,0.08000,0.0,0.0,0.0000,0.0,0.160000,0.0
1,David Ansen,1,114709.0,Newsweek,A winning animated feature that has something ...,2008-08-18,9559.0,Toy story,13,a winning animated feature that has something ...,...,0.384615,0.0,0.00000,0.00000,0.0,0.0,0.0000,0.0,0.230769,0.0
2,Leonard Klady,1,114709.0,Variety,The film sports a provocative and appealing st...,2008-06-09,9559.0,Toy story,17,the film sports a provocative and appealing st...,...,0.277778,0.0,0.00000,0.00000,0.0,0.0,0.0000,0.0,0.166667,0.0
3,Jonathan Rosenbaum,1,114709.0,Chicago Reader,"An entertaining computer-generated, hyperreali...",2008-03-10,9559.0,Toy story,14,an entertaining computer generated hyperrealis...,...,0.437500,0.0,0.00000,0.00000,0.0,0.0,0.0625,0.0,0.125000,0.0
4,Michael Booth,1,114709.0,Denver Post,"As Lion King did before it, Toy Story revived ...",2007-05-03,9559.0,Toy story,40,as lion king did before it toy story revived t...,...,0.341463,0.0,0.02439,0.02439,0.0,0.0,0.0000,0.0,0.146341,0.0


In [85]:
sen.head()

,pos,word,pos_score,neg_score,objectivity_score
0,adj,.22-caliber,0.0,0.0,1.0
1,adj,.22-calibre,0.0,0.0,1.0
2,adj,.22_caliber,0.0,0.0,1.0
3,adj,.22_calibre,0.0,0.0,1.0
4,adj,.38-caliber,0.0,0.0,1.0


In [86]:
sen_dict = {'ADJ': {}, 'NOUN': {}, 'VERB': {}, 'ADV': {}}
# sen_dict = {'adj': {}, 'noun': {}, 'verb': {}, 'adv': {}}
for row in tqdm(sen.itertuples()):
#     print(row[1], row[2])
    sen_dict[row[1].upper()] [row[2]] = {'objectivity': row[5], 'pos_vs_neg': row[6]}

0it [00:00, ?it/s]


IndexError: tuple index out of range

In [ ]:
sen_dict.keys()

In [ ]:
def scorer(parsed):
    obj_scores, pvn_scores = [], []
    for token in [t for t in parsed if t.pos_ in ['NOUN','VERB','ADV','ADJ']]:
        try:
            obj_scores.append(sen_dict[token.pos_][str(token)]['objectivity'])
            pvn_scores.append(sen_dict[token.pos_][str(token)]['pos_vs_neg'])
        except:
            pass
    if len(obj_scores) == 0:
        obj_scores = [1.]
    if len(pvn_scores) == 0:
        pvn_scores = [0.]
    return [obj_scores, pvn_scores]


scores = []
for i, parsed in tqdm(enumerate(parsed_quotes)):
    scores.append(scorer(parsed))
    
rt['objectivity_avg'] = [np.mean(x[0]) for x in scores]
rt['pos_vs_neg_avg'] = [np.mean(x[1]) for x in scores]

In [ ]:
rt.head()

In [ ]:
rt_desc=rt.sort_values('pos_vs_neg_avg',ascending=False)